In [1]:
import pandas as pd
import numpy as np
import biosppy.signals.ecg as ecg
import math

from sklearn import preprocessing
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer

In [2]:
def load_data():
    X_train = pd.read_csv("X_train.csv")
    X_test = pd.read_csv("X_test.csv")
    y_train = pd.read_csv("y_train.csv")

    #dropping id column
    X_train = X_train.drop('id', axis = 1)
    X_test = X_test.drop('id', axis = 1)
    y_train = y_train.drop('id', axis = 1)
    
    #reshuffling data
    X_train['y'] = y_train
    X_train = X_train.sample(frac=1).reset_index(drop=True)
    y_train = X_train['y']
    X_train = X_train.drop('y', axis = 1)
    
    return X_train, X_test, y_train

In [3]:
xtrain, xtest, ytrain = load_data()

Goal in code below

Repeat for all training examples:  
    First extract r peaks from signal with ssf_segmenter (or any other function that works)  
    Then use that together with the signal to get heartbeats  
    Heartbeat function outputs an n x m array where n are the amount of heartbeats and m (=180) are the features per     heartbeat  
    Compute mean, median, standard deviation, max and min over all the heartbeat features (=> 5*180 = 900 features) and insert     in dataframe  

In [7]:
rpeaks

array([ 136,  213,  238,  262,  286,  298,  301,  307,  341,  427,  603,
        627,  649,  677,  732,  810,  912,  951,  957, 1032, 1069, 1176,
       1257, 1338, 1567, 1651, 1872, 1888, 1954, 2034, 2049, 2131, 2172,
       2254, 2273, 2452, 2472, 2496, 2581, 2809, 2835, 2918, 3160, 3241,
       3472, 3487, 3553, 3722, 3749, 3767, 3771, 3849, 3916, 4063, 4142,
       4287, 4323, 4366, 4368, 4448, 4625, 4638, 4667, 4746, 4767, 4967,
       5047, 5221, 5265, 5345, 5396, 5438, 5498, 5517, 5539, 5558, 5561,
       5638, 5724, 5801, 5845, 5926, 6089, 6138, 6141, 6219, 6318, 6397,
       6418, 6439, 6518, 6583, 6604, 6681, 6714, 6718, 6795, 6922, 6966,
       7044, 7045, 7165, 7210, 7227, 7233, 7261, 7307, 7373, 7427, 7448,
       7466, 7508, 7589, 7679, 7711, 7724, 7785, 7813, 7895, 7912, 8040,
       8105, 8125, 8146, 8163, 8229, 8397, 8427, 8472, 8552])

In [9]:
# Create an empty data frame with 180 * 5 columns
training_set = pd.DataFrame(columns = np.arange(900))

for index, row in xtrain.iterrows():
    
    signal_array = row.values
    rpeaks = ecg.ssf_segmenter(signal_array, sampling_rate = 300)
    rpeaks = np.array(rpeaks).ravel()
    
    rpeaks = ecg.correct_rpeaks(signal = signal_array, rpeaks = rpeaks, sampling_rate = 300, tol = 0.01)
    rpeaks = np.array(rpeaks).ravel()
    heartbeat, rpeaks = ecg.extract_heartbeats(signal = signal_array, 
                                          rpeaks = rpeaks, sampling_rate = 300)
    
    # Iterate through all the heartbeat features and compute mean, median and standard deviation
    means = np.array([])
    medians = np.array([])
    stds = np.array([])
    maximums = np.array([])
    minimums = np.array([])
    for feat in heartbeat.T:
        means = np.append(means, np.mean(feat))
        medians = np.append(medians, np.median(feat))
        stds = np.append(stds, np.std(feat))
        maximums = np.append(maximums, max(feat))
        minimums = np.append(minimums, min(feat))
    
    # Add as training example
    values = np.append(means, medians)
    values = np.append(values, stds)
    values = np.append(values, maximums)
    values = np.append(values, minimums)
    training_set.loc[index] = values
    
        

In [10]:
pd.set_option('display.max_columns', 900)
training_set.head(10)

0           1           2           3          4          5    \
0  -56.985612  -61.363309  -64.913669  -67.690647 -69.291367 -69.748201   
1   27.637931   26.954023   23.890805   16.821839   4.557471 -14.706897   
2   80.984076   75.187898   67.496815   58.270701  48.429936  38.197452   
3   16.173203    7.026144   -6.212418  -12.542484   1.169935  16.464052   
4 -102.860000 -112.066667 -110.593333 -100.346667 -84.960000 -67.533333   
5  -24.305882  -24.176471  -24.129412  -24.023529 -23.611765 -22.958824   
6 -158.526316 -151.552632 -135.934211 -117.052632 -96.921053 -77.276316   
7    1.250000    2.521739    4.771739    8.244565  11.967391  15.451087   
8  -33.750000  -33.294118  -32.911765  -32.500000 -31.823529 -30.955882   
9   -7.101587  -12.361905  -17.495238  -22.514286 -27.123810 -30.917460   

         6          7           8           9           10          11   \
0 -68.956835 -67.068345  -64.471223  -61.521583  -58.241007  -54.823741   
1 -41.948276 -77.862069 -120.902299 -167.545977 -213.356322 -254.040230   
2  28.493631  19.703822   12.152866    5.522293    0.296178   -2.977707   
3  24.058824  11.813725   -3.758170  -11.366013    1.866013   18.944444   
4 -51.480000 -38.266667  -28.546667  -22.120000  -17.593333  -14.660000   
5 -21.864706 -20.564706  -19.270588  -17.817647  -16.429412  -15.652941   
6 -59.697368 -44.697368  -32.105263  -21.197368  -11.092105   -2.210526   
7  18.456522  20.750000   22.027174   21.771739   20.918478   19.695652   
8 -30.088235 -29.397059  -28.882353  -28.529412  -28.235294  -27.867647   
9 -34.092063 -37.006349  -39.882540  -42.730159  -46.009524  -50.025397   

          12          13          14          15          16          17   \
0  -51.255396  -47.827338  -44.776978  -42.061151  -39.543165  -37.284173   
1 -284.241379 -301.074713 -304.362069 -296.206897 -278.965517 -257.718391   
2   -4.436306   -4.490446   -3.101911   -1.130573    0.885350    2.589172   
3   29.287582   19.290850    3.689542   -7.235294    0.735294   10.895425   
4  -12.400000  -10.140000   -7.820000   -5.740000   -3.866667   -1.993333   
5  -15.241176  -15.335294  -15.664706  -15.829412  -15.547059  -14.794118   
6    5.868421   13.105263   19.736842   26.644737   33.539474   39.986842   
7   18.309783   17.260870   15.961957   14.755435   13.684783   13.543478   
8  -27.676471  -27.014706  -26.191176  -24.941176  -22.838235  -20.176471   
9  -54.685714  -59.885714  -65.901587  -72.761905  -80.244444  -86.847619   

          18          19          20          21          22         23   \
0  -35.169065  -33.035971  -31.014388  -29.000000  -27.125899 -25.352518   
1 -232.793103 -206.505747 -179.132184 -152.103448 -125.816092 -99.557471   
2    3.958599    4.955414    5.372611    4.949045    3.847134   2.601911   
3   14.084967   -3.094771  -24.107843  -37.284314  -28.598039 -14.143791   
4   -0.040000    1.780000    3.386667    4.800000    6.246667   7.973333   
5  -13.682353  -12.641176  -11.905882  -11.452941  -11.264706 -11.276471   
6   46.131579   52.065789   56.736842   60.881579   62.473684  60.894737   
7   13.684783   14.027174   15.130435   17.320652   19.940217  22.663043   
8  -16.735294  -12.852941   -8.691176   -4.367647    0.338235   5.132353   
9  -92.361905  -96.152381  -97.482540  -95.777778  -91.307937 -84.241270   

         24         25         26         27         28         29   \
0 -23.906475 -22.978417 -22.471223 -22.205036 -21.953237 -21.453237   
1 -75.183908 -52.586207 -31.695402 -14.126437  -0.183908  10.068966   
2   1.114650   0.101911   0.315287   1.375796   3.789809   7.006369   
3  -4.124183 -12.094771 -22.660131 -27.029412 -11.460784   7.196078   
4   9.773333  11.486667  13.100000  14.513333  15.813333  17.186667   
5 -11.411765 -11.488235 -11.300000 -10.976471 -10.611765 -10.288235   
6  57.250000  50.447368  41.947368  32.039474  21.723684  11.526316   
7  24.396739  24.483696  23.326087  21.271739  18.711957  16.385870   
8   8.926471  11.617647  13.308824

In [11]:
training_set.dropna(axis = 1).shape

(5117, 540)

In [12]:
steps = [("impute", SimpleImputer()), 
         ("scaler", preprocessing.StandardScaler()), ("classifier", SVC())]

pipeline = Pipeline(steps = steps)

parameters = {"impute__strategy": ["mean", "median", "constant"],
              "impute__fill_value": [0],
              "classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
              "classifier__class_weight": ["balanced"]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1_micro')

grid.fit(training_set.values, ytrain.values.ravel())

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('impute',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='mean',
                                                      verbose=0)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('classifier',
                                        SVC(C=1.0, cache_size=200,
                                            class_

In [15]:
grid.best_score_

0.5813953488372093

In [14]:
grid.best_params_

{'classifier__C': 50,
 'classifier__class_weight': 'balanced',
 'classifier__gamma': 'auto',
 'classifier__kernel': 'rbf',
 'impute__strategy': 'median'}

In [16]:
xtrain.shape

(5117, 17813)